In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import SGD
from keras.applications import VGG16
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_gen.flow_from_directory('/home/jebastin/Desktop/data/Train', target_size = (160,160),
                                       batch_size=10, class_mode='binary')
valid = valid_gen.flow_from_directory('/home/jebastin/Desktop/data/Valid', target_size = (160,160),
                                       batch_size=10, class_mode='binary')

In [ ]:
model = Sequential()

model.add(Conv2D(8, 3, activation='relu', input_shape=(160,160,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
sgd1 = SGD(lr=9e-5, decay=1e-6, momentum=0.99)
model.compile(optimizer=sgd1, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train, steps_per_epoch=22, epochs=50, validation_data=valid,
                              validation_steps=4) 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(160,160,3))

for layer in base_model.layers:
        layer.trainable = False

In [ ]:
model_final = Sequential()

model_final.add(base_model)
model_final.add(Flatten())
model_final.add(Dense(1024, activation="relu"))
model_final.add(Dropout(0.4))
model_final.add(Dense(256, activation="relu"))
model_final.add(Dense(1, activation="sigmoid"))

In [ ]:
sgd2 = SGD(lr=1e-4, decay=1e-6, momentum=0.9)
model_final.compile(optimizer=sgd2, loss='binary_crossentropy', metrics=['accuracy'])
model_final.fit_generator(train, steps_per_epoch=22, epochs=25, validation_data=valid, validation_steps=4)

In [ ]:
train.reset()
valid.reset()
 
for layer in base_model.layers[15:]:
    layer.trainable = True

In [ ]:
model_final.compile(optimizer=sgd2, loss='binary_crossentropy', metrics=['accuracy'])
hist = model_final.fit_generator(train, steps_per_epoch=22, epochs=25, validation_data=valid, validation_steps=4)

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()